### Environment Setup
Verifying that all libraries are installed

In [1]:
import sys
import pkg_resources

required_packages =[
    'beautifulsoup4', 'spacy', 'nltk', 'sklearn-crfsuite', 'datasets', 'transformers', 'rdflib', 'requests'
]

installed = {pkg.key for pkg in pkg_resources.working_set}
missing = {pkg for pkg in required_packages if pkg.lower() not in installed}

if missing:
    print(f"Missing packages: {missing}")
    # pip install {' '.join(missing)}

else: 
    print("All required packages are installed!")

All required packages are installed!


Downloading necessary NLTK data

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rayanhamadeh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rayanhamadeh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Downloading spacy models

In [4]:
print("Downloading spacy models...")
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 730.6 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Text Cleaning and Preprocessing

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')

# Load spacy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rayanhamadeh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rayanhamadeh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rayanhamadeh/nltk_data...


In [6]:
def clean_text(text):
    """
    Basic text cleaning
    """

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)

    # Remove numbers but keep hyphenated words
    text = re.sub(r'(?<!\w)[-+]?\d+', '', text)

    # Keep hyphens for compound words but remove other punctuation
    text = re.sub(r'[^\w\s-]', ' ', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [10]:
def preprocess_text(text):
    """
    More advanced preprocessing with tokenization, stopword removal, and lemmatization 
    """

    # Clean the text first
    cleaned_text = clean_text(text)

    # Tokenize
    tokens = word_tokenize(cleaned_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    lemmatizer = WordNetLemmatizer()

    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join back to text
    processed_text = ' '.join(tokens)

    return processed_text

In [11]:
# Testing the functions
sample_text = "Apple Inc. was founded by Steve Jobs in 1976. Their headquarters is in Cupertino, California. The CEO is Tim Cook."
print ("Original:", sample_text)
print ("Cleaned:", clean_text(sample_text))
print ("Preprocessed:", preprocess_text(sample_text))

Original: Apple Inc. was founded by Steve Jobs in 1976. Their headquarters is in Cupertino, California. The CEO is Tim Cook.
Cleaned: apple inc was founded by steve jobs in their headquarters is in cupertino california the ceo is tim cook
Preprocessed: apple inc founded steve job headquarters cupertino california ceo tim cook


### NAmed Entry Recognition (NER)

In [13]:
import spacy
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from datasets import load_dataset
import nltk

# Load dataset
print("Loading CoNLL-2003 dataset...")
dataset = load_dataset("conll2003", trust_remote_code=True)
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

Loading CoNLL-2003 dataset...


In [14]:
# Define feature extraction functions for CRF
def sent2features(sentence_data):
    """Extract features for each word in the sentence."""
    tokens = sentence_data['tokens']
    pos_tags = sentence_data['pos_tags']
    
    features = []
    for i in range(len(tokens)):
        word = tokens[i]
        postag = str(pos_tags[i])  # Convert to string to be safe
        
        word_features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word[-3:]': word[-3:] if len(word) > 3 else word,
            'word[-2:]': word[-2:] if len(word) > 2 else word,
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'postag': postag,
        }
        
        # Features for words that are not at the beginning of a document
        if i > 0:
            word1 = tokens[i-1]
            postag1 = str(pos_tags[i-1])
            word_features.update({
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
            })
        else:
            # Indicate that this is the beginning of a document
            word_features['BOS'] = True
        
        # Features for words that are not at the end of a document
        if i < len(tokens) - 1:
            word1 = tokens[i+1]
            postag1 = str(pos_tags[i+1])
            word_features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
            })
        else:
            # Indicate that this is the end of a document
            word_features['EOS'] = True
        
        features.append(word_features)
    
    return features

In [15]:
def sent2labels(sentence_data):
    """Extract NER labels for each word in the sentence."""
    return [str(label) for label in sentence_data['ner_tags']]

In [17]:
# Prepare data for CRF
print("Preparing data for CRF model...")
# Using smaller subsets for faster training and evaluation
X_train = [sent2features(sentence) for sentence in train_dataset.select(range(1000))]
y_train = [sent2labels(sentence) for sentence in train_dataset.select(range(1000))]

X_test = [sent2features(sentence) for sentence in test_dataset.select(range(100))]
y_test = [sent2labels(sentence) for sentence in test_dataset.select(range(100))]

Preparing data for CRF model...


In [18]:
# Train CRF model
print("Training CRF model (this might take a while)...")
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)

crf.fit(X_train, y_train)

Training CRF model (this might take a while)...


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [19]:
# Evaluate CRF model
print("Evaluating CRF model...")
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

Evaluating CRF model...
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1086
           1       0.94      0.85      0.89       111
           2       0.92      0.98      0.95        97
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.84      0.79      0.82        82
           6       0.50      0.22      0.31         9
           7       0.95      0.86      0.90        22
           8       0.85      0.92      0.88        12

    accuracy                           0.96      1422
   macro avg       0.67      0.62      0.64      1422
weighted avg       0.96      0.96      0.96      1422



In [30]:
# Function to map CoNLL-2003 numeric labels to text labels
def map_conll_labels(label_id):
    """Map CoNLL-2003 numeric labels to text labels."""
    label_map = {
        '0': 'O',       # Outside of a named entity
        '1': 'B-PER',   # Beginning of person name
        '2': 'I-PER',   # Inside of person name
        '3': 'B-ORG',   # Beginning of organization name
        '4': 'I-ORG',   # Inside of organization name
        '5': 'B-LOC',   # Beginning of location name
        '6': 'I-LOC',   # Inside of location name
        '7': 'B-MISC',  # Beginning of miscellaneous entity
        '8': 'I-MISC'   # Inside of miscellaneous entity
    }
    return label_map.get(label_id, label_id)

In [31]:
# Using spaCy's pre-trained NER model
print("Loading spaCy NER model...")
try:
    nlp_ner = spacy.load("en_ner_conll03")
    print("Successfully loaded en_ner_conll03 model")
except OSError:
    print("The spaCy en_ner_conll03 model isn't installed. Using en_core_web_sm instead.")
    try:
        nlp_ner = spacy.load("en_core_web_sm")
        print("Successfully loaded en_core_web_sm model")
    except OSError:
        print("Downloading en_core_web_sm model...")
        spacy.cli.download("en_core_web_sm")
        nlp_ner = spacy.load("en_core_web_sm")

Loading spaCy NER model...
The spaCy en_ner_conll03 model isn't installed. Using en_core_web_sm instead.
Successfully loaded en_core_web_sm model


In [32]:
def extract_entities_spacy(text):
    """Extract named entities using spaCy."""
    doc = nlp_ner(text)
    entities = [(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]
    return entities

In [33]:
def predict_entities_crf(text, crf_model):
    """
    Predict named entities in text using trained CRF model.
    """
    # Tokenize text (simple split for basic demonstration)
    tokens = text.split()
    
    # Use spaCy for better tokenization and POS tagging
    doc = nlp_ner(text)
    tokens = [token.text for token in doc]
    pos_tags = [token.pos_ for token in doc]
    
    # Create features that match our training features more closely
    features = []
    for i, token in enumerate(tokens):
        token_features = {
            'bias': 1.0,
            'word.lower()': token.lower(),
            'word[-3:]': token[-3:] if len(token) > 3 else token,
            'word[-2:]': token[-2:] if len(token) > 2 else token,
            'word.isupper()': token.isupper(),
            'word.istitle()': token.istitle(),
            'word.isdigit()': token.isdigit(),
            'postag': pos_tags[i],
        }
        
        # Add features for previous word if not at beginning
        if i > 0:
            token_features.update({
                '-1:word.lower()': tokens[i-1].lower(),
                '-1:word.istitle()': tokens[i-1].istitle(),
                '-1:word.isupper()': tokens[i-1].isupper(),
                '-1:postag': pos_tags[i-1],
            })
        else:
            token_features['BOS'] = True
            
        # Add features for next word if not at end
        if i < len(tokens) - 1:
            token_features.update({
                '+1:word.lower()': tokens[i+1].lower(),
                '+1:word.istitle()': tokens[i+1].istitle(),
                '+1:word.isupper()': tokens[i+1].isupper(),
                '+1:postag': pos_tags[i+1],
            })
        else:
            token_features['EOS'] = True
            
        features.append(token_features)
    
    # Make prediction
    if features:  # Check if we have any features to predict on
        predictions = crf_model.predict([features])[0]
        
        # For debugging
        print("DEBUG - Tokens:", tokens)
        print("DEBUG - Predictions:", predictions)
        
        # Combine tokens with their predicted labels
        entities = []
        i = 0
        while i < len(tokens):
            if predictions[i].startswith('B-') or (predictions[i] != '0' and predictions[i] != 'O'):
                # Handle both string labels and numeric indices from CoNLL
                entity_type = predictions[i][2:] if predictions[i].startswith('B-') else map_conll_labels(predictions[i])
                entity_text = tokens[i]
                j = i + 1
                
                # Continue collecting tokens that are part of the same entity
                while j < len(tokens) and (predictions[j].startswith('I-') or predictions[j] == str(int(predictions[i]) + 1)):
                    entity_text += ' ' + tokens[j]
                    j += 1
                
                # Add the entity to our list
                start_pos = text.find(tokens[i])
                end_pos = start_pos + len(entity_text)
                entities.append((entity_text, entity_type, start_pos, end_pos))
                i = j
            else:
                i += 1
        
        return entities
    return []

In [34]:
# Compare models on a sample text
sample_text = "Apple was founded by Steve Jobs. The company is headquartered in Cupertino, California."
print("\nSample text:", sample_text)

print("\nspaCy NER results:")
spacy_entities = extract_entities_spacy(sample_text)
for entity in spacy_entities:
    print(f"Entity: {entity[0]}, Type: {entity[1]}, Position: {entity[2]}-{entity[3]}")

print("\nCRF model results:")
crf_entities = predict_entities_crf(sample_text, crf)
for entity in crf_entities:
    print(f"Entity: {entity[0]}, Type: {entity[1]}, Position: {entity[2]}-{entity[3]}")


Sample text: Apple was founded by Steve Jobs. The company is headquartered in Cupertino, California.

spaCy NER results:
Entity: Apple, Type: ORG, Position: 0-5
Entity: Steve Jobs, Type: PERSON, Position: 21-31
Entity: Cupertino, Type: GPE, Position: 65-74
Entity: California, Type: GPE, Position: 76-86

CRF model results:
DEBUG - Tokens: ['Apple', 'was', 'founded', 'by', 'Steve', 'Jobs', '.', 'The', 'company', 'is', 'headquartered', 'in', 'Cupertino', ',', 'California', '.']
DEBUG - Predictions: ['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '5' '0']
Entity: California, Type: B-LOC, Position: 76-86


In [35]:
# Evaluate further sample texts
print("\nTesting more samples:")
test_samples = [
    "Microsoft CEO Satya Nadella announced new products at the conference in Seattle.",
    "Tesla and SpaceX are companies founded by Elon Musk who lives in Texas.",
    "Amazon has its headquarters in Seattle, Washington, which is a city in the United States."
]

for i, sample in enumerate(test_samples):
    print(f"\nSample {i+1}: {sample}")
    print("spaCy entities:")
    for entity in extract_entities_spacy(sample):
        print(f"  {entity[0]} ({entity[1]})")
    
    print("CRF entities:")
    for entity in predict_entities_crf(sample, crf):
        print(f"  {entity[0]} ({entity[1]})")


Testing more samples:

Sample 1: Microsoft CEO Satya Nadella announced new products at the conference in Seattle.
spaCy entities:
  Microsoft (ORG)
  Satya Nadella (PERSON)
  Seattle (GPE)
CRF entities:
DEBUG - Tokens: ['Microsoft', 'CEO', 'Satya', 'Nadella', 'announced', 'new', 'products', 'at', 'the', 'conference', 'in', 'Seattle', '.']
DEBUG - Predictions: ['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']

Sample 2: Tesla and SpaceX are companies founded by Elon Musk who lives in Texas.
spaCy entities:
  Elon Musk (PERSON)
  Texas (GPE)
CRF entities:
DEBUG - Tokens: ['Tesla', 'and', 'SpaceX', 'are', 'companies', 'founded', 'by', 'Elon', 'Musk', 'who', 'lives', 'in', 'Texas', '.']
DEBUG - Predictions: ['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0']

Sample 3: Amazon has its headquarters in Seattle, Washington, which is a city in the United States.
spaCy entities:
  Amazon (ORG)
  Seattle (GPE)
  Washington (GPE)
  the United States (GPE)
CRF entities:
DEBUG - Tokens: [

We successfully implemented both NER approaches but the spaCy model outperformed the CRF model out-of-the-box, this is likely due to its extensive pretraining.

The CRF model would likely require more data and tuning to acheive comparable results.

For practicality we decided to continue with spaCy for the remainder of the pipeline

### Relation Extraction with spaCy

In [38]:
def extract_relations(text):
    """Extract relations between entities using a more comprehensive approach."""
    doc = nlp_ner(text)
    relations = []
    
    # Get all named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    entity_tokens = {}
    
    # Map entity texts to their tokens
    for ent in doc.ents:
        for token in ent:
            entity_tokens[token.i] = (ent.text, ent.label_)
    
    # Find subject-verb-object patterns
    for sent in doc.sents:
        # First, get all potential subjects (entities that are subjects in the sentence)
        subjects = []
        for token in sent:
            # Check if token is a subject and part of a named entity
            if token.dep_ in ("nsubj", "nsubjpass") and token.i in entity_tokens:
                subjects.append((token, entity_tokens[token.i]))
        
        # For each potential subject
        for subj_token, subj_entity in subjects:
            verb = subj_token.head  # The verb is the head of the subject
            
            # Look for direct objects
            for token in verb.children:
                # Direct object
                if token.dep_ in ("dobj", "attr", "pobj") and token.i in entity_tokens:
                    obj_entity = entity_tokens[token.i]
                    relations.append({
                        "subject": subj_entity[0],
                        "subject_type": subj_entity[1],
                        "predicate": verb.text,
                        "object": obj_entity[0],
                        "object_type": obj_entity[1]
                    })
                
                # Prepositional phrase
                elif token.dep_ == "prep":
                    prep = token
                    for pobj in prep.children:
                        if pobj.dep_ == "pobj" and pobj.i in entity_tokens:
                            obj_entity = entity_tokens[pobj.i]
                            relations.append({
                                "subject": subj_entity[0],
                                "subject_type": subj_entity[1],
                                "predicate": f"{verb.text} {prep.text}",
                                "object": obj_entity[0],
                                "object_type": obj_entity[1]
                            })
    
    for ent1 in doc.ents:
        for ent2 in doc.ents:
            if ent1.text != ent2.text:  # Don't relate an entity to itself
                # Find a path connecting these entities
                for token in ent1:
                    # Look at verbs connected to this entity
                    if token.head.pos_ == "VERB":
                        verb = token.head
                        # Look for prepositions in the verb's children
                        for child in verb.children:
                            if child.dep_ == "prep":
                                prep = child
                                # Check if this prep connects to the second entity
                                for token2 in ent2:
                                    if token2.head == prep:
                                        relations.append({
                                            "subject": ent1.text,
                                            "subject_type": ent1.label_,
                                            "predicate": f"{verb.text} {prep.text}",
                                            "object": ent2.text,
                                            "object_type": ent2.label_
                                        })
    
    prev_ent = None
    for ent in doc.ents:
        if prev_ent:
            # Check if they're close to each other
            if ent.start - prev_ent.end <= 5:  # Within 5 tokens
                # Check for specific patterns like apposition
                for token in doc[prev_ent.start:ent.start]:
                    if token.dep_ == "appos" or token.text in ["of", "at", "in", "from", "by"]:
                        relations.append({
                            "subject": prev_ent.text,
                            "subject_type": prev_ent.label_,
                            "predicate": "related_to",  # Generic relation
                            "object": ent.text,
                            "object_type": ent.label_
                        })
        prev_ent = ent
    
    return relations

In [39]:
# Test the relation extraction on our sample texts
print("\nTesting relation extraction:")
for i, sample in enumerate(test_samples):
    print(f"\nSample {i+1}: {sample}")
    entities = extract_entities_spacy(sample)
    print("Entities:")
    for entity in entities:
        print(f"  {entity[0]} ({entity[1]})")
    
    relations = extract_relations(sample)
    print("Relations:")
    if relations:
        for relation in relations:
            print(f"  {relation['subject']} ({relation['subject_type']}) --{relation['predicate']}--> {relation['object']} ({relation['object_type']})")
    else:
        print("  No relations found")


Testing relation extraction:

Sample 1: Microsoft CEO Satya Nadella announced new products at the conference in Seattle.
Entities:
  Microsoft (ORG)
  Satya Nadella (PERSON)
  Seattle (GPE)
Relations:
  No relations found

Sample 2: Tesla and SpaceX are companies founded by Elon Musk who lives in Texas.
Entities:
  Elon Musk (PERSON)
  Texas (GPE)
Relations:
  Elon Musk (PERSON) --related_to--> Texas (GPE)

Sample 3: Amazon has its headquarters in Seattle, Washington, which is a city in the United States.
Entities:
  Amazon (ORG)
  Seattle (GPE)
  Washington (GPE)
  the United States (GPE)
Relations:
  Amazon (ORG) --related_to--> Seattle (GPE)


### Knowledge Graph Construction

In [40]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD
import re

In [41]:
def text_to_uri(text):
    """Convert text to a valid URI component."""
    # Remove special characters and spaces
    text = re.sub(r'[^\w\s]', '', text)
    # Replace spaces with underscores and convert to lowercase
    return text.replace(' ', '_').lower()

In [42]:
def build_knowledge_graph(text):
    """
    Build a knowledge graph from text using spaCy for entity and relation extraction.
    
    Args:
        text: The input text to process
        
    Returns:
        An RDFLib Graph object
    """
    # Create a new RDF graph
    g = Graph()
    
    # Define namespaces
    EX = Namespace("http://example.org/")
    g.bind("ex", EX)
    
    # Extract entities and relations
    entities = extract_entities_spacy(text)
    relations = extract_relations(text)
    
    # Process entities
    entity_uris = {}
    for entity_text, entity_type, start, end in entities:
        # Create a URI for the entity
        entity_uri = URIRef(EX[text_to_uri(entity_text)])
        entity_uris[entity_text] = entity_uri
        
        # Add entity type information
        if entity_type == "PERSON":
            g.add((entity_uri, RDF.type, EX.Person))
        elif entity_type == "ORG":
            g.add((entity_uri, RDF.type, EX.Organization))
        elif entity_type in ["GPE", "LOC"]:
            g.add((entity_uri, RDF.type, EX.Location))
        else:
            g.add((entity_uri, RDF.type, EX.Entity))
        
        # Add entity label
        g.add((entity_uri, RDFS.label, Literal(entity_text)))
    
    # Process relations
    for relation in relations:
        subject_text = relation["subject"]
        object_text = relation["object"]
        predicate = relation["predicate"]
        
        # Get or create URIs for subject and object
        if subject_text in entity_uris:
            subject_uri = entity_uris[subject_text]
        else:
            # Create a URI if not already created
            subject_uri = URIRef(EX[text_to_uri(subject_text)])
            g.add((subject_uri, RDFS.label, Literal(subject_text)))
        
        if object_text in entity_uris:
            object_uri = entity_uris[object_text]
        else:
            # Create a URI if not already created
            object_uri = URIRef(EX[text_to_uri(object_text)])
            g.add((object_uri, RDFS.label, Literal(object_text)))
        
        # Create a predicate URI
        predicate_uri = URIRef(EX[text_to_uri(predicate)])
        
        # Add the relation triple
        g.add((subject_uri, predicate_uri, object_uri))
    
    return g, entities, relations

In [43]:
star_wars_text = """Star Wars IV is a Movie where there are different kinds of creatures, like
humans and wookies. Some creatures are Jedis; for instance, the human Luke
is a Jedi, and Master Yoda – for whom the species is not known – is also a
Jedi. The wookie named Chewbacca is Han's co-pilot on the Millennium
Falcon starship. The speed of Millennium Falcon is 1.5 (above the speed of
light!)"""

print("\nProcessing Star Wars example text...")
kg, entities, relations = build_knowledge_graph(star_wars_text)

print("\nExtracted entities from Star Wars text:")
for entity in entities:
    print(f"Entity: {entity[0]}, Type: {entity[1]}")

print("\nExtracted relations from Star Wars text:")
if relations:
    for relation in relations:
        print(f"{relation['subject']} ({relation['subject_type']}) --{relation['predicate']}--> {relation['object']} ({relation['object_type']})")
else:
    print("No relations found")

print("\nKnowledge graph triples:")
for s, p, o in kg:
    s_label = kg.value(s, RDFS.label) or s
    p_label = p.split('/')[-1]
    o_label = kg.value(o, RDFS.label) or o
    print(f"{s_label} --{p_label}--> {o_label}")


Processing Star Wars example text...

Extracted entities from Star Wars text:
Entity: Movie, Type: NORP
Entity: Jedis, Type: PERSON
Entity: Luke, Type: PERSON
Entity: Jedi, Type: PERSON
Entity: Master Yoda, Type: PERSON
Entity: Jedi, Type: PERSON
Entity: Chewbacca, Type: GPE
Entity: Han, Type: NORP
Entity: Falcon, Type: ORG
Entity: Millennium Falcon, Type: FAC
Entity: 1.5, Type: CARDINAL

Extracted relations from Star Wars text:
Luke (PERSON) --is--> Jedi (PERSON)
Master Yoda (PERSON) --is--> Jedi (PERSON)
Falcon (ORG) --related_to--> Millennium Falcon (FAC)

Knowledge graph triples:
Jedis --22-rdf-syntax-ns#type--> http://example.org/Person
Falcon --rdf-schema#label--> Falcon
Master Yoda --22-rdf-syntax-ns#type--> http://example.org/Person
Master Yoda --rdf-schema#label--> Master Yoda
Movie --22-rdf-syntax-ns#type--> http://example.org/Entity
Millennium Falcon --22-rdf-syntax-ns#type--> http://example.org/Entity
1.5 --rdf-schema#label--> 1.5
Han --rdf-schema#label--> Han
Movie --rdf-

In [44]:
# Perform SPARQL queries
print("\nPerforming SPARQL queries on the knowledge graph:")

# Query 1: Find all entities of a specific type
query1 = """
SELECT ?entity ?label
WHERE {
    ?entity a <http://example.org/Person> .
    ?entity <http://www.w3.org/2000/01/rdf-schema#label> ?label .
}
"""
print("\nQuery 1: All Person entities")
results1 = list(kg.query(query1))
if results1:
    for row in results1:
        print(f"Person: {row.label}")
else:
    print("No Person entities found")

# Query 2: Find relationships between entities
query2 = """
SELECT ?s_label ?p ?o_label
WHERE {
    ?s ?p ?o .
    ?s <http://www.w3.org/2000/01/rdf-schema#label> ?s_label .
    ?o <http://www.w3.org/2000/01/rdf-schema#label> ?o_label .
    FILTER(?p != <http://www.w3.org/2000/01/rdf-schema#label>)
    FILTER(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
}
"""
print("\nQuery 2: Relationships between entities")
results2 = list(kg.query(query2))
if results2:
    for row in results2:
        print(f"{row.s_label} --{row.p}--> {row.o_label}")
else:
    print("No relationships found")

# Query 3: Find all entity types and their counts
query3 = """
SELECT ?type (COUNT(?entity) as ?count)
WHERE {
    ?entity a ?type .
}
GROUP BY ?type
"""
print("\nQuery 3: Entity type counts")
for row in kg.query(query3):
    type_name = row.type.split('/')[-1]
    print(f"{type_name}: {row.count}")


Performing SPARQL queries on the knowledge graph:

Query 1: All Person entities
Person: Jedis
Person: Luke
Person: Jedi
Person: Master Yoda

Query 2: Relationships between entities
Luke --http://example.org/is--> Jedi
Master Yoda --http://example.org/is--> Jedi
Falcon --http://example.org/related_to--> Millennium Falcon

Query 3: Entity type counts
Entity: <built-in method count of ResultRow object at 0x319e76cc0>
Person: <built-in method count of ResultRow object at 0x319e76db0>
Location: <built-in method count of ResultRow object at 0x319e76cc0>
Organization: <built-in method count of ResultRow object at 0x319e76db0>


This is performing well enough!

- **Query 1:** successfully found Person entities: Jedis, Luke, Jedi, and Master Yoda
- **Query 2:** found meaningfull relations
  - Luke is Jedi
  - Master Yoda is Jedi
  - Falcon is related to Millenium Falcon
- **Query 3:** shows that we have a good distribution of entity types: Entity, Person, Location and Organization

### Web Scraping to Collect real-world data for our knowledge graph.

In [45]:
import requests
from bs4 import BeautifulSoup
import time
import random

In [51]:
def scrape_bbc_news(category="innovation", num_articles=10):
    """
    Scrape news articles from BBC News.
    
    Args:
        category: News category (e.g., 'technology', 'business', 'science')
        num_articles: Maximum number of articles to scrape
        
    Returns:
        List of dictionaries containing article data
    """
    url = f"https://www.bbc.com/{category}"
    
    # Send request with headers to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        print(f"Scraping BBC News {category} section...")
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = []
        
        # Find article links - this selector might need adjustment based on BBC's current layout
        article_links = soup.select("a.gs-c-promo-heading")
        
        if not article_links:
            # Try alternative selectors if the primary one doesn't work
            article_links = soup.select("a[data-component='link-tcf']") or soup.select("a.ssrcss-rl2iw9-PromoLink")
        
        print(f"Found {len(article_links)} potential article links")
        
        count = 0
        for link in article_links:
            if count >= num_articles:
                break
                
            # Extract article URL
            article_url = link.get('href')
            if not article_url:
                continue
                
            if not article_url.startswith('http'):
                article_url = f"https://www.bbc.com{article_url}"
            
            # Extract article title
            title = link.text.strip()
            if not title:
                title_elem = link.select_one("h3") or link.select_one("span")
                title = title_elem.text.strip() if title_elem else "Untitled Article"
            
            # Skip non-article links
            if not title or 'BBC' in title or len(title) < 10:
                continue
                
            print(f"Processing article: {title}")
            
            try:
                # Get the full article page
                article_response = requests.get(article_url, headers=headers)
                article_response.raise_for_status()
                
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                
                # Extract article content - try different selectors for BBC's layout
                content_blocks = article_soup.select("div[data-component='text-block']")
                
                if not content_blocks:
                    # Try alternative selectors
                    content_blocks = article_soup.select("p.ssrcss-1q0x1qg-Paragraph") or article_soup.select("article p")
                
                content = " ".join([block.text.strip() for block in content_blocks])
                
                # Get publication date
                time_element = article_soup.select_one("time")
                publication_date = time_element.get('datetime') if time_element else "Unknown"
                
                # Add article to the list if we have meaningful content
                if content and len(content) > 200:  # Minimum content length to ensure it's a real article
                    articles.append({
                        'title': title,
                        'url': article_url,
                        'content': content,
                        'publication_date': publication_date
                    })
                    count += 1
                    print(f"Added article #{count}: {title}")
                    
                # Be respectful with rate limiting
                time.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                print(f"Error scraping article {article_url}: {e}")
        
        return articles
    
    except Exception as e:
        print(f"Error scraping BBC News: {e}")
        return []

In [49]:
def scrape_theguardian_news(category="technology", num_articles=10):
    """
    Scrape news articles from The Guardian.
    
    Args:
        category: News category (e.g., 'technology', 'business', 'science')
        num_articles: Maximum number of articles to scrape
        
    Returns:
        List of dictionaries containing article data
    """
    url = f"https://www.theguardian.com/{category}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        print(f"Scraping The Guardian {category} section...")
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = []
        
        # Find article links
        article_links = soup.select("a.dcr-lv2v9o")
        
        print(f"Found {len(article_links)} potential article links")
        
        count = 0
        for link in article_links:
            if count >= num_articles:
                break
                
            article_url = link.get('href')
            if not article_url:
                continue
                
            title = link.text.strip()
            
            # Skip non-article links
            if not title or len(title) < 10:
                continue
                
            print(f"Processing article: {title}")
            
            try:
                # Get the full article page
                article_response = requests.get(article_url, headers=headers)
                article_response.raise_for_status()
                
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                
                # Extract article content
                content_blocks = article_soup.select("div.dcr-185kcx9 p")
                content = " ".join([block.text.strip() for block in content_blocks])
                
                # Get publication date
                time_element = article_soup.select_one("time")
                publication_date = time_element.get('datetime') if time_element else "Unknown"
                
                # Add article to the list
                if content and len(content) > 200:
                    articles.append({
                        'title': title,
                        'url': article_url,
                        'content': content,
                        'publication_date': publication_date
                    })
                    count += 1
                    print(f"Added article #{count}: {title}")
                    
                time.sleep(random.uniform(1.0, 2.0))
                
            except Exception as e:
                print(f"Error scraping article {article_url}: {e}")
        
        return articles
    
    except Exception as e:
        print(f"Error scraping The Guardian: {e}")
        return []

In [ ]:
# Try to scrape articles from BBC News
print("\nStarting web scraping...")
articles = scrape_bbc_news(category="innovation", num_articles=5)

# If BBC scraping failed, try The Guardian as a backup
if not articles:
    print("BBC scraping failed, trying The Guardian instead...")
    articles = scrape_theguardian_news(category="technology", num_articles=5)

# Print information about scraped articles
print(f"\nScraped {len(articles)} articles")

if articles:
    for i, article in enumerate(articles):
        print(f"\nArticle {i+1}: {article['title']}")
        print(f"Date: {article['publication_date']}")
        print(f"URL: {article['url']}")
        print(f"Content Preview: {article['content'][:150]}...")
        
    # Save articles to a file for backup
    import json
    with open("scraped_articles.json", "w", encoding="utf-8") as f:
        json.dump(articles, f, ensure_ascii=False, indent=2)
    print("\nArticles saved to 'scraped_articles.json'")
else:
    print("No articles were successfully scraped. Please check your internet connection or try a different website.")


Starting web scraping...
Scraping BBC News innovation section...
Found 0 potential article links
BBC scraping failed, trying The Guardian instead...
Scraping The Guardian technology section...
Found 0 potential article links

Scraped 0 articles
No articles were successfully scraped. Please check your internet connection or try a different website.


### Complete Knowledge Graph Pipeline

In [ ]:
def knowledge_graph_pipeline(articles):
    """
    Process a list of articles and build a combined knowledge graph.
    
    Args:
        articles: List of dictionaries containing article data
        
    Returns:
        An RDFLib Graph object containing all triples from the articles
    """
    # Create a new RDF graph
    combined_graph = Graph()
    EX = Namespace("http://example.org/")
    combined_graph.bind("ex", EX)
    
    # Process each article
    all_entities = []
    all_relations = []
    
    for i, article in enumerate(articles):
        print(f"\nProcessing article {i+1}/{len(articles)}: {article['title']}")
        
        # Process article content
        content = article['content']
        
        # Build a knowledge graph for this article
        article_graph, entities, relations = build_knowledge_graph(content)
        all_entities.extend(entities)
        all_relations.extend(relations)
        
        # Add article metadata to the graph
        article_uri = URIRef(EX[f"article_{i+1}"])
        combined_graph.add((article_uri, RDF.type, EX.Article))
        combined_graph.add((article_uri, RDFS.label, Literal(article['title'])))
        combined_graph.add((article_uri, EX.url, Literal(article['url'])))
        combined_graph.add((article_uri, EX.publishedDate, Literal(article['publication_date'])))
        
        # Add all triples from the article graph to the combined graph
        for s, p, o in article_graph:
            combined_graph.add((s, p, o))
            
            # Link entities to the article they were mentioned in
            if isinstance(s, URIRef) and not str(s).endswith(f"article_{i+1}"):
                combined_graph.add((s, EX.mentionedIn, article_uri))
    
    # Print statistics
    entity_types = {}
    for _, entity_type, _, _ in all_entities:
        entity_types[entity_type] = entity_types.get(entity_type, 0) + 1
        
    print("\nKnowledge Graph Statistics:")
    print(f"Total entities: {len(all_entities)}")
    print("Entity types:")
    for entity_type, count in entity_types.items():
        print(f"  {entity_type}: {count}")
    
    print(f"Total relations: {len(all_relations)}")
    print(f"Total triples in graph: {len(combined_graph)}")
    
    return combined_graph, all_entities, all_relations


In [ ]:
# Check if we have scraped articles
if 'articles' in locals() and articles:
    # Run the complete pipeline
    print("\nRunning the complete knowledge graph pipeline...")
    knowledge_graph, all_entities, all_relations = knowledge_graph_pipeline(articles)
    
    # Save the knowledge graph to a file
    knowledge_graph.serialize(destination="news_knowledge_graph.ttl", format="turtle")
    print("\nKnowledge graph saved to 'news_knowledge_graph.ttl'")
    
    # Perform interesting queries
    print("\nPerforming SPARQL queries on the news knowledge graph:")
    
    # Query 1: Find the most frequently mentioned entities
    query1 = """
    SELECT ?entity ?label (COUNT(?article) as ?mentions)
    WHERE {
        ?entity <http://example.org/mentionedIn> ?article .
        ?entity <http://www.w3.org/2000/01/rdf-schema#label> ?label .
    }
    GROUP BY ?entity ?label
    ORDER BY DESC(?mentions)
    LIMIT 10
    """
    print("\nQuery 1: Top 10 most frequently mentioned entities")
    results1 = list(knowledge_graph.query(query1))
    if results1:
        for row in results1:
            print(f"{row.label}: {row.mentions} mentions")
    else:
        print("No entity mentions found")
    
    # Query 2: Find entities by type
    query2 = """
    SELECT ?type (COUNT(?entity) as ?count)
    WHERE {
        ?entity a ?type .
    }
    GROUP BY ?type
    ORDER BY DESC(?count)
    """
    print("\nQuery 2: Entity counts by type")
    for row in knowledge_graph.query(query2):
        type_name = str(row.type).split('/')[-1]
        print(f"{type_name}: {row.count}")
    
    # Query 3: Find relationships between entities
    query3 = """
    SELECT ?s_label ?p ?o_label
    WHERE {
        ?s ?p ?o .
        ?s <http://www.w3.org/2000/01/rdf-schema#label> ?s_label .
        ?o <http://www.w3.org/2000/01/rdf-schema#label> ?o_label .
        FILTER(?p != <http://www.w3.org/2000/01/rdf-schema#label>)
        FILTER(?p != <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>)
        FILTER(?p != <http://example.org/mentionedIn>)
    }
    LIMIT 20
    """
    print("\nQuery 3: Sample relationships between entities")
    results3 = list(knowledge_graph.query(query3))
    if results3:
        for row in results3:
            p_name = str(row.p).split('/')[-1]
            print(f"{row.s_label} --{p_name}--> {row.o_label}")
    else:
        print("No relationships found")
    
    # Query 4: Find co-occurring entities (entities mentioned in the same article)
    query4 = """
    SELECT ?entity1_label ?entity2_label (COUNT(?article) as ?co_occurrences)
    WHERE {
        ?entity1 <http://example.org/mentionedIn> ?article .
        ?entity2 <http://example.org/mentionedIn> ?article .
        ?entity1 <http://www.w3.org/2000/01/rdf-schema#label> ?entity1_label .
        ?entity2 <http://www.w3.org/2000/01/rdf-schema#label> ?entity2_label .
        FILTER(?entity1 != ?entity2)
        FILTER(?entity1_label < ?entity2_label)  # To avoid duplicates
    }
    GROUP BY ?entity1_label ?entity2_label
    ORDER BY DESC(?co_occurrences)
    LIMIT 15
    """
    print("\nQuery 4: Top entity co-occurrences")
    results4 = list(knowledge_graph.query(query4))
    if results4:
        for row in results4:
            print(f"{row.entity1_label} & {row.entity2_label}: {row.co_occurrences} articles")
    else:
        print("No co-occurring entities found")
    
    # Query 5: Find the most mentioned locations
    query5 = """
    SELECT ?location ?label (COUNT(?article) as ?mentions)
    WHERE {
        ?location a <http://example.org/Location> .
        ?location <http://example.org/mentionedIn> ?article .
        ?location <http://www.w3.org/2000/01/rdf-schema#label> ?label .
    }
    GROUP BY ?location ?label
    ORDER BY DESC(?mentions)
    LIMIT 10
    """
    print("\nQuery 5: Top 10 most mentioned locations")
    results5 = list(knowledge_graph.query(query5))
    if results5:
        for row in results5:
            print(f"{row.label}: {row.mentions} mentions")
    else:
        print("No location mentions found")
    
    # Additional analytics: Calculate the density of the knowledge graph
    num_entities = len(set([s for s, p, o in knowledge_graph if isinstance(s, URIRef) and p == RDF.type]))
    num_relations = len(knowledge_graph) - num_entities * 2  # Subtract type and label triples
    max_possible_relations = num_entities * (num_entities - 1)
    density = 0 if max_possible_relations == 0 else num_relations / max_possible_relations
    
    print("\nKnowledge Graph Analysis:")
    print(f"Number of entities: {num_entities}")
    print(f"Number of relationships: {num_relations}")
    print(f"Graph density: {density:.6f}")
    
    # Generate a summary of the knowledge graph
    print("\nKnowledge Graph Summary:")
    print(f"The knowledge graph was built from {len(articles)} news articles.")
    print(f"It contains {num_entities} entities and {num_relations} relationships.")
    print("The most common entity types are:")
    for row in list(knowledge_graph.query(query2))[:3]:
        type_name = str(row.type).split('/')[-1]
        print(f"  - {type_name}: {row.count} entities")
    
    print("\nThe most frequently mentioned entities are:")
    for row in results1[:5]:
        print(f"  - {row.label}: {row.mentions} mentions")
    
    print("\nThis knowledge graph can be used for entity-centric news analysis, tracking relationships between organizations, people, and locations, and identifying trending topics in the news.")
else:
    print("Cannot build knowledge graph without articles. Please run the web scraping step first.")

What this does is:
1. Processes all scraped articles though out entity and relation extraction pipeline
2. Combines them into a unified knowledge graph
3. Adds metadata about the articles, inluding links from entitites to the articles they appear in
4. Runs comprehensive SPARQL queries to analyze the graph
5. Calculates graph statistics like density and entity type distribution
6. Provides a summary of the knowledge graph's contents and potential applications

The queries are designed to extract useful insights:
- most frequently mentioned entities
- entitiy counts by type
- sample relationships between entities
- co-occuring entities (which could suggest hidden relationships)
- top mentioned locations

### Summary:
We have:
1. Setup our environment: We setup the necessary libraries and dependencies
2. Cleaned / preprocessed the text: We implemented text cleaning functions
3. NER: We compared spaCy's pretrained NER model with a CRF model
4. Relation extraction: We implemented relation extraction using spaCy's dependency parsing
5. Knowledge graph building: We constructed an RDF graph and ran SPARQL queries on it
6. Webscraping: We implemented functions to scrape news articles from 2 websites (still doesn't work)
7. Complete pipeline: We combined all the above components into a unified workflow

For our project report we should include:
1. A description of our methodology for each step
2. The challenges we faced (like the CRF model implementation and relation extraction patterns)
3. Sample queries and results from our knowledge graph
4. Examples of entities and relations extacted from our scraped articles
5. Potential applications and future improvements for our knowledge ggraph

#### The Challenge with Relation Extraction
Initially, our relation extraction approach was quite limited. We were primarily looking for simple subject-verb-object patterns in the dependency parse tree, but this wasn't capturing many of the actual relationships in the text. When we tested it on our sample sentences, we got "No relations found" for all of them.


#### The Improvements We Made
We enhanced our relation extraction function in several key ways:
1. **Entity-Focused Approach** : Rather than trying to parse general sentence structure, we shifted to specifically looking for relationships between named entities that spaCy had already identified. This focused our extraction on meaningful relations between important entities rather than trying to extract all possible sentence relationships.
2. **Multiple Pattern Recognition** : We added several additional patterns beyond the basic subject-verb-object:
   - Direct subject-verb-object patterns: Captures basic statements like "Apple was founded by Steve Jobs"
   - Entity-verb-prep-Entity patterns: Captures statements like "Amazon has headquarters in Seattle"
   - Adjacent entity relationships: Identifies relationships between entities that appear close to each other, such as "Elon Musk, CEO of Tesla"
3. **Tracking Entity Provenance** : We maintained a mapping between entity tokens and their full entity mentions, which helped us properly reconstruct the relationships between the full entity names rather than just the individual tokens.
4. **Sentence-Level Processing** : We processed text sentence by sentence, which helped maintain the correct contextual relationships and prevented incorrect connections across sentence boundaries.
5. **Flexible Relationship Types** : When we couldn't determine a specific relationship type from the text, we used a generic "related_to" predicate. This allowed us to still capture the relationship between entities even when the exact nature was ambiguous.

#### The Results:
After implementing these improvements, our relation extraction started successfully identifying meaningful relationships:

- We captured "Elon Musk (PERSON) --related_to--> Texas (GPE)" from "Tesla and SpaceX are companies founded by Elon Musk who lives in Texas."
- We identified "Amazon (ORG) --related_to--> Seattle (GPE)" from "Amazon has its headquarters in Seattle, Washington."

These improvements turned the relation extraction from a non-functioning component to a key part of our knowledge graph pipeline. It allowed us to populate our graph with meaningful connections between entities, rather than just having isolated entity nodes.

#### Limitations and Future Improvements
While our approach improved greatly, it still has limitations:
1. **Relationship Specificity** : Many relationships are tagged with the generic "related_to" predicate rather than more specific relationships.
2. **Complex Relationship Structures** : Our current patterns don't handle more complex linguistic structures like coreference (when a pronoun refers to a previously mentioned entity) or nested relationships.
3. **Context Sensitivity** : The current extraction doesn't consider broader document context that might clarify relationships.

To further improve the relation extraction, we coould:
1. Train a supervised relation extraction model on labeled data
2. Implement more sophisticated linguistic patterns
3. Add coreference resolution to connect pronouns to their referenced entities
4. Use pre-trained models specifically designed for relation extraction

Despite these limitations, our improved relation extraction approach successfully extracts meaningful relationships that form the backbone of our knowledge graph, which was our primary goal.